![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# TextMatcherInternal

In this notebook, we will examine the `TextMatcherInternal` annotator and its model version `TextMatcherInternalModel`.

This annotator match exact phrases provided in a file against a
Document.


**📖 Learning Objectives:**

1. Understand how to match exact phrases by using pre-defined dictionary.

2. Become comfortable using the different parameters of the annotator.

**🔗 Helpful Links:**

For extended examples of usage, see the [Spark NLP Workshop Repository](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/42.TextMatcher.ipynb)

Reference Documentation: [TextMatcherInternal](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#textmatcherinternal)


## **🎬 Colab Setup**

In [ ]:
import json
import os

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.1

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [ ]:
import json
import os

from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SparkSession

import sparknlp
import sparknlp_jsl

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.pretrained import ResourceDownloader
from pyspark.sql import functions as F

import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

import string
import numpy as np

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(secret = license_keys["SECRET"], params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 5.3.0
Spark NLP_JSL Version : 5.3.0


## **🖨️ Input/Output Annotation Types**
- Input: ``DOCUMENT`` , ``TOKEN``    
- Output: ``CHUNK``

## **🔎 Parameters**


- `setEntities` *(str)*: Sets the external resource for the entities.
        path : str
            Path to the external resource
        read_as : str, optional
            How to read the resource, by default ReadAs.TEXT
        options : dict, optional
            Options for reading the resource, by default {"format": "text"}
- `setCaseSensitive` *(Boolean)*: Sets whether to match regardless of case. (Default: True)

- `setMergeOverlapping` *(Boolean)*:Sets whether to merge overlapping matched chunks. (Default: False)

- `setEntityValue` *(str)*: Sets the value for the entity metadata field. If any entity value isn't set in the file, we need to set it for the entity value.

- `setBuildFromTokens` *(Boolean)*:  Sets whether the TextMatcherInternal should take the CHUNK from TOKEN.

- `setDelimiter` *(str)*:  Sets Value for the delimiter between Phrase, Entity.





## How to Use `TextMatcherInternal`

First of all, we should create a source file that includes all the chunks or tokens we need to capture. In the example below, we use `#` as a delimiter to separate the label and entity. So we need to set parameter like this `setDelimiter('#')`.

In [ ]:
matcher_drug = """
Aspirin 100mg#Drug
aspirin#Drug
paracetamol#Drug
amoxicillin#Drug
ibuprofen#Drug
lansoprazole#Drug
"""

with open ('matcher_drug.csv', 'w') as f:
  f.write(matcher_drug)

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

entityExtractor = TextMatcherInternal()\
    .setInputCols(["document", "token"])\
    .setEntities("matcher_drug.csv")\
    .setOutputCol("matched_text")\
    .setCaseSensitive(False)\
    .setDelimiter("#")\
    .setMergeOverlapping(True)

mathcer_pipeline = Pipeline().setStages([
                  documentAssembler,
                  tokenizer,
                  entityExtractor])

data = spark.createDataFrame([["John's doctor prescribed aspirin 100mg for his heart condition, along with paracetamol for his fever, amoxicillin for his tonsilitis, ibuprofen for his inflammation, and lansoprazole for his GORD."]]).toDF("text")

matcher_model = mathcer_pipeline.fit(data)
result = matcher_model.transform(data)

In [ ]:
result.select(F.explode(F.arrays_zip(
              result.matched_text.result,
              result.matched_text.begin,
              result.matched_text.end,
              result.matched_text.metadata,)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias('label')).show(truncate=70)

+-------------+-----+---+-----+
|        chunk|begin|end|label|
+-------------+-----+---+-----+
|aspirin 100mg|   25| 37| Drug|
|  paracetamol|   75| 85| Drug|
|  amoxicillin|  102|112| Drug|
|    ibuprofen|  134|142| Drug|
| lansoprazole|  170|181| Drug|
+-------------+-----+---+-----+



As you see above mather_drug file includes 2 similar entities aspirin and aspirin 100mg and our text includes both of them. So if you want to see both of them you need to set `MergeOverlapping` parameter as `False`. You can look at the below example.

In [ ]:
entityExtractor = TextMatcherInternal()\
    .setInputCols(["document", "token"])\
    .setEntities("matcher_drug.csv") \
    .setOutputCol("matched_text")\
    .setCaseSensitive(False)\
    .setDelimiter("#")\
    .setMergeOverlapping(False)

mathcer_pipeline = Pipeline().setStages([
                  documentAssembler,
                  tokenizer,
                  entityExtractor])

matcher_model = mathcer_pipeline.fit(data)
result = matcher_model.transform(data)

In [ ]:
result.select(F.explode(F.arrays_zip(
              result.matched_text.result,
              result.matched_text.begin,
              result.matched_text.end,
              result.matched_text.metadata,)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias('label')).show(truncate=70)

+-------------+-----+---+-----+
|        chunk|begin|end|label|
+-------------+-----+---+-----+
|      aspirin|   25| 31| Drug|
|aspirin 100mg|   25| 37| Drug|
|  paracetamol|   75| 85| Drug|
|  amoxicillin|  102|112| Drug|
|    ibuprofen|  134|142| Drug|
| lansoprazole|  170|181| Drug|
+-------------+-----+---+-----+



When we set the `CaseSensitive` parameter to `True`, it means we're considering the case sensitivity of chunks in the source file. Consequently, some chunks may not be visible due to differences in their case compared to the source file.

In [ ]:
entityExtractor = TextMatcherInternal()\
    .setInputCols(["document", "token"])\
    .setEntities("matcher_drug.csv") \
    .setOutputCol("matched_text")\
    .setCaseSensitive(True)\
    .setDelimiter("#")\
    .setMergeOverlapping(False)

mathcer_pipeline = Pipeline().setStages([
                  documentAssembler,
                  tokenizer,
                  entityExtractor])

matcher_model = mathcer_pipeline.fit(data)
result = matcher_model.transform(data)

In [ ]:
result.select(F.explode(F.arrays_zip(
              result.matched_text.result,
              result.matched_text.begin,
              result.matched_text.end,
              result.matched_text.metadata,)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias('label')).show(truncate=70)

+------------+-----+---+-----+
|       chunk|begin|end|label|
+------------+-----+---+-----+
|     aspirin|   25| 31| Drug|
| paracetamol|   75| 85| Drug|
| amoxicillin|  102|112| Drug|
|   ibuprofen|  134|142| Drug|
|lansoprazole|  170|181| Drug|
+------------+-----+---+-----+



## Multiple Entities

We can set multiple entities in the same file.

In [ ]:
multiple_entites= """
Aspirin 100mg#Drug
paracetamol#Drug
amoxicillin#Drug
ibuprofen#Drug
lansoprazole#Drug
fever#Symptom
headache#Symptom
tonsilitis#Disease
GORD#Disease
heart condition#Disease
"""

with open ('multiple_entities.csv', 'w') as f:
  f.write(multiple_entites)

In [ ]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

entityExtractor = TextMatcherInternal() \
    .setInputCols(["document", "token"]) \
    .setEntities("multiple_entities.csv") \
    .setOutputCol("matched_text")\
    .setCaseSensitive(False)\
    .setDelimiter("#")

mathcer_pipeline = Pipeline().setStages([
                  documentAssembler,
                  tokenizer,
                  entityExtractor])

data = spark.createDataFrame([["John's doctor prescribed aspirin 100mg for his heart condition, along with paracetamol for his fever and headache, amoxicillin for his tonsilitis, ibuprofen for his inflammation, and lansoprazole for his GORD."]]).toDF("text")

matcher_model = mathcer_pipeline.fit(data)
result = matcher_model.transform(data)

In [ ]:
result.select(F.explode(F.arrays_zip(
              result.matched_text.result,
              result.matched_text.begin,
              result.matched_text.end,
              result.matched_text.metadata,)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias('label')).show(truncate=70)

+---------------+-----+---+-------+
|          chunk|begin|end|  label|
+---------------+-----+---+-------+
|heart condition|   47| 61|Disease|
|     tonsilitis|  135|144|Disease|
|           GORD|  204|207|Disease|
|          fever|   95| 99|Symptom|
|       headache|  105|112|Symptom|
|  aspirin 100mg|   25| 37|   Drug|
|    paracetamol|   75| 85|   Drug|
|    amoxicillin|  115|125|   Drug|
|      ibuprofen|  147|155|   Drug|
|   lansoprazole|  183|194|   Drug|
+---------------+-----+---+-------+



## `TextMatcherInternalModel`

This annotator is an instantiated model of the `TextMatcherInternal`. Once you build an `TextMatcherInternal()`, you can save it and use it with `TextMatcherInternalModel()` via `load()` function. <br/>

Let's re-build one of examples that we have done before and save it.

In [ ]:
entityExtractor = TextMatcherInternal() \
    .setInputCols(["document", "token"]) \
    .setEntities("matcher_drug.csv") \
    .setOutputCol("matched_text")\
    .setCaseSensitive(False)\
    .setDelimiter("#")

mathcer_pipeline = Pipeline().setStages([
                  documentAssembler,
                  tokenizer,
                  entityExtractor])

data = spark.createDataFrame([["John's doctor prescribed aspirin 100mg for his heart condition, along with paracetamol for his fever and headache, amoxicillin for his tonsilitis, ibuprofen for his inflammation, and lansoprazole for his GORD."]]).toDF("text")

matcher_model = mathcer_pipeline.fit(data)
result = matcher_model.transform(data)

Saving the approach to disk.

In [ ]:
matcher_model.stages[-1].write().overwrite().save("matcher_internal_model")

Loading the saved model and using it with the `TextMatcherInternalModel()` via `load`.

In [ ]:
entity_ruler = TextMatcherInternalModel.load('/content/matcher_internal_model') \
    .setInputCols(["document", "token"]) \
    .setOutputCol("matched_text")\
    .setCaseSensitive(False)\
    .setDelimiter("#")

pipeline = Pipeline(stages=[documentAssembler,
                            tokenizer,
                            entity_ruler])

pipeline_model = pipeline.fit(data)
result = pipeline_model.transform(data)

Checking the result

In [ ]:
result.select(F.explode(F.arrays_zip(
              result.matched_text.result,
              result.matched_text.begin,
              result.matched_text.end,
              result.matched_text.metadata,)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias('label')).show(truncate=70)

+-------------+-----+---+-----+
|        chunk|begin|end|label|
+-------------+-----+---+-----+
|      aspirin|   25| 31| Drug|
|aspirin 100mg|   25| 37| Drug|
|  paracetamol|   75| 85| Drug|
|  amoxicillin|  115|125| Drug|
|    ibuprofen|  147|155| Drug|
| lansoprazole|  183|194| Drug|
+-------------+-----+---+-----+



As seen above, we built an `TextMatcherInternal`, saved it and used the saved model with `TextMatcherInternalModel`.

## Using LightPipeline

The TextMatcherInternal annotator can also be applied by using LightPipeline:

In [ ]:
light_pipeline = LightPipeline(pipeline_model)

In [ ]:
annotations = light_pipeline.fullAnnotate("John's doctor prescribed aspirin 100mg for his heart condition, along with paracetamol for his fever and headache, amoxicillin for his tonsilitis, ibuprofen for his inflammation, and lansoprazole for his GORD.")[0]
annotations.keys()

dict_keys(['document', 'token', 'matched_text'])

In [ ]:
annotations.get('matched_text')

[Annotation(chunk, 25, 31, aspirin, {'entity': 'Drug', 'sentence': '0', 'chunk': '0'}, []),
 Annotation(chunk, 25, 37, aspirin 100mg, {'entity': 'Drug', 'sentence': '0', 'chunk': '1'}, []),
 Annotation(chunk, 75, 85, paracetamol, {'entity': 'Drug', 'sentence': '0', 'chunk': '2'}, []),
 Annotation(chunk, 115, 125, amoxicillin, {'entity': 'Drug', 'sentence': '0', 'chunk': '3'}, []),
 Annotation(chunk, 147, 155, ibuprofen, {'entity': 'Drug', 'sentence': '0', 'chunk': '4'}, []),
 Annotation(chunk, 183, 194, lansoprazole, {'entity': 'Drug', 'sentence': '0', 'chunk': '5'}, [])]

Display the result with `spark-nlp-display`.

In [ ]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

visualiser.display(annotations, label_col='matched_text')